In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report
from pyod.models.iforest import IForest
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import OneSidedSelection
import os
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from pyod.models.pca import PCA
from sklearn.mixture import GaussianMixture
from sklearn.covariance import EllipticEnvelope
from sklearn.model_selection import GridSearchCV

import os, sys, pickle
import argparse, sys

import seaborn as sns
sns.set_style('white')

import warnings
warnings.filterwarnings('ignore')



# parser = argparse.ArgumentParser()
# parser.add_argument('-label', help=' : Please set the label') 
# args = parser.parse_args()


df = pd.read_csv('data/modified_dataset.csv')


# def remove_outliers(df, column_name, factor=1.5):
#     Q1 = df[column_name].quantile(0.25)
#     Q3 = df[column_name].quantile(0.75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     df_filtered = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]
#     return df_filtered

# # 모든 칼럼에 대해 이상치 제거 실행
# for column_name in df.columns:
#     df = remove_outliers(df, column_name)







df_stroke_0 = df[df['Cancer'] == 0]
df_stroke_1 = df[df['Cancer'] == 1]



#train, test data processing 1,2,3,4중 선택

#1.정상으로만 학습, 테스트는 정상과 이상 반반
answer_label = 'Cancer'
X = df[df.columns.difference([answer_label])]
df_normal = df[df[answer_label] == 0]
df_abnormal = df[df[answer_label] == 1]
test_normal_df = df_normal.sample(n=len(df_stroke_1), random_state = 0)
test_df = pd.concat([df_abnormal, test_normal_df])
X_test = test_df[test_df.columns.difference([answer_label])]
y_test = test_df[answer_label]
train_df = df_normal.drop(test_normal_df.index)
X_train = train_df[train_df.columns.difference([answer_label])]
y_train = train_df[answer_label]



In [2]:
from sklearn.svm import OneClassSVM
def main_task(kernel, nu):
    ocsvm = OneClassSVM(kernel=kernel, nu = nu, verbose = True)
    ocsvm.fit(X_train)
    ocsvm_test_pred = ocsvm.predict(X_test)
    ocsvm_test_pred = pd.DataFrame(ocsvm_test_pred)
    ocsvm_test_pred = ocsvm_test_pred.replace({-1:1, 1:0})
    return round(f1_score(y_test, ocsvm_test_pred),3)

In [5]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

def optuna_main(trial: optuna.Trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'])
    nu = trial.suggest_float('nu', 0.01, 1.00, step = 0.02)
    print('kernel : ', kernel)
    print('nu : ', nu)

    f1_score = main_task(kernel = kernel, nu = nu)

    print('<f1_score:', f1_score,'>')
    return f1_score

study = optuna.create_study(direction='maximize')
study.optimize(optuna_main, n_trials=300)
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_params : {study.best_trial.params}')

[I 2023-11-08 14:43:33,967] A new study created in memory with name: no-name-9b3ee10e-11ec-4ea0-bc74-41cb58bc73d8


kernel :  poly
nu :  0.25
[LibSVM]..
*.
*.
*

[I 2023-11-08 14:43:36,326] Trial 0 finished with value: 0.031 and parameters: {'kernel': 'poly', 'nu': 0.25}. Best is trial 0 with value: 0.031.


.
*
optimization finished, #iter = 3325
obj = 762.923815, rho = 1.243314
nSV = 2674, nBSV = 2657
<f1_score: 0.031 >
kernel :  poly
nu :  0.75
[LibSVM]..
*.
*
optimization finished, #iter = 2506
obj = 173590.939973, rho = 106.852912
nSV = 8003, nBSV = 7991


[I 2023-11-08 14:43:39,299] Trial 1 finished with value: 0.259 and parameters: {'kernel': 'poly', 'nu': 0.75}. Best is trial 1 with value: 0.259.


...

[I 2023-11-08 14:44:45,215] Trial 11 finished with value: 0.792 and parameters: {'kernel': 'sigmoid', 'nu': 0.37}. Best is trial 10 with value: 0.8.


<f1_score: 0.792 >
kernel :  sigmoid
nu :  0.05
[LibSVM]

[I 2023-11-08 14:44:45,681] Trial 12 finished with value: 0.428 and parameters: {'kernel': 'sigmoid', 'nu': 0.05}. Best is trial 10 with value: 0.8.
[W 2023-11-08 14:44:45,691] Trial 13 failed with parameters: {'kernel': 'precomputed', 'nu': 0.35000000000000003} because of the following error: ValueError('Precomputed matrix must be a square matrix. Input is a 10663x5 matrix.').
Traceback (most recent call last):
  File "/home/mohee/.local/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_11715/3985630578.py", line 11, in optuna_main
    f1_score = main_task(kernel = kernel, nu = nu)
  File "/tmp/ipykernel_11715/3548912541.py", line 4, in main_task
    ocsvm.fit(X_train)
  File "/home/mohee/.local/lib/python3.10/site-packages/sklearn/svm/_classes.py", line 1814, in fit
    super().fit(X, np.ones(_num_samples(X)), sample_weight=sample_weight)
  File "/home/mohee/.local/lib/python3.10/site-packages/skl

*
optimization finished, #iter = 431
obj = -31762.924855, rho = -94.067111
nSV = 534, nBSV = 532
<f1_score: 0.428 >
kernel :  precomputed
nu :  0.35000000000000003


ValueError: Precomputed matrix must be a square matrix. Input is a 10663x5 matrix.

In [11]:
#결과
print(f'study.best_trial.value : {study.best_trial.value}')
print(f'study.best_paramse : {study.best_params}')

study.best_trial.value : 0.801
study.best_paramse : {'contamination': 0.15000000000000002, 'n_bins': 10}
